In [54]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [55]:
df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
df1["proprosion"] = np.where(df1["cant_viewed_product"]>0, (df1["cant_checkouts"]/df1["cant_viewed_product"]),0)


In [56]:
df2 = pd.read_csv('data/vistas_promedio.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('data/featureUsuarioRealizaAlMenos10EventosEn20Minutos.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('data/features_mas_vistos.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2).fillna(0)
df_unidos = df_unidos.join(df3).fillna(0)
df_unidos = df_unidos.join(df4).fillna(0)
#df_unidos = df_unidos.drop("Unnamed: 0",1)
df_unidos.head()

,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_ad_campaign_hit,ad_campaign_hit,lead,cant_brand_listing,brand listing,total_sesiones,promedio_eventos_por_sesion,proprosion,mas_prom,conte_10,vio_mas_apple,vio_mas_samsung,vio_mas_motorola,vio_mas_lg
person,,,,,,,,,,,,,,,,,,
4886f805,0.0,1.0,4.0,1.0,0.0,False,False,0.0,False,1,9.000000,0.250000,False,True,False,True,False,False
ad93850f,0.0,1.0,20.0,0.0,10.0,True,False,15.0,True,5,13.000000,0.050000,False,True,True,False,False,False
0297fc1e,0.0,7.0,404.0,6.0,29.0,True,True,4.0,True,89,6.370787,0.017327,True,True,True,False,False,False
2d681dd8,0.0,1.0,13.0,1.0,1.0,True,False,5.0,True,2,13.000000,0.076923,False,False,False,False,True,False
cccea85e,0.0,1.0,739.0,1.0,15.0,True,False,7.0,True,17,49.176471,0.001353,True,True,False,False,True,False


In [57]:
labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')


df_test = df1.join(labels, how= "inner")
df_test.head()

,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_ad_campaign_hit,ad_campaign_hit,lead,cant_brand_listing,brand listing,total_sesiones,promedio_eventos_por_sesion,proprosion,label
person,,,,,,,,,,,,,
ad93850f,0.0,1.0,20.0,0.0,10.0,True,False,15.0,True,5,13.000000,0.050000,0
1b9f7cf6,0.0,1.0,9.0,8.0,0.0,False,False,3.0,True,3,8.333333,0.111111,0
de8fe91b,0.0,1.0,27.0,13.0,0.0,False,False,3.0,True,2,26.500000,0.037037,0
45baf068,0.0,1.0,6.0,0.0,5.0,True,False,3.0,True,1,20.000000,0.166667,0
99abca5a,2.0,18.0,630.0,5.0,12.0,True,True,30.0,True,49,16.714286,0.028571,0


In [58]:
X, y = df_test.iloc[:,:-1],df_test.iloc[:,-1]

In [59]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [68]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 150, alpha = 20, n_estimators = 1000)

In [69]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=20, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=150, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [70]:
preds = xg_reg.predict(X_test)

In [71]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.220783


In [72]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df1, how='inner')

In [91]:
"""kaggle_pred = xg_reg.predict(df_events)
proba_de_comprar = [x for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = abs(series.values)"""


kaggle_pred = preds.predict_proba(df_events)
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

AttributeError: 'numpy.ndarray' object has no attribute 'predict_proba'

In [88]:
df_submit.to_csv('submit_xg.csv')

In [89]:
df_submit

,label
person,
4886f805,0.005848
0297fc1e,0.176950
2d681dd8,0.081366
cccea85e,0.009690
4c8a8b93,0.066271
29ebb414,0.070730
3dc1950f,0.127456
8ea4c165,0.077113
d8cfe234,0.208068
